In [ ]:
# ✅ Instala solo lo necesario
!pip install --quiet langdetect tqdm

# 📚 Librerías
import pandas as pd
import requests
import random
import time
from langdetect import detect
from tqdm import tqdm
from datetime import datetime
# 📦 Archivos por categoría
archivos = {
    "Palestina": "narrativas_palestina_clusterizadas.csv",
    "Israel": "narrativas_israel_clusterizadas.csv",
    "Internacional": "narrativas_internacional_clusterizadas.csv",
    "Internacional Medio Oriente": "narrativas_internacional_medio_oriente_clusterizadas.csv"
}

# 📊 Cargar resumen
df_resumen = pd.read_csv("resumen_topicos_por_categoria.csv")

# 📓 Cargar log incremental (comentarios ya guardados)
try:
    log = pd.read_csv("log_comentarios_extraidos.csv")
    print("✅ Log cargado con", len(log), "comentarios.")
except FileNotFoundError:
    log = pd.DataFrame(columns=["comment_id", "video_id", "categoria", "topic", "fecha", "text"])
    print("📁 Log creado desde cero.")
# 🔐 Lista de claves
API_KEYS = [
]
def extraer_comentarios(video_id, categoria, topic, max_results=40):
    global API_KEYS, log
    comentarios = []

    for api_key in API_KEYS:
        url = f"https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId={video_id}&maxResults={max_results}&textFormat=plainText&key={api_key}"
        try:
            response = requests.get(url)
            data = response.json()
            if 'items' not in data:
                continue
            for item in data['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                comment_id = item['id']
                text = comment['textDisplay']
                fecha = comment['publishedAt'][:10]

                # 🔁 evitar repetidos
                if comment_id in log["comment_id"].values:
                    continue

                # 🚫 evitar spam
                if len(text) < 20 or any(x in text.lower() for x in ['http', '@', '#', 'subscribe']):
                    continue

                try:
                    if detect(text) != "en":
                        continue
                except:
                    continue

                comentarios.append({
                    "comment_id": comment_id,
                    "video_id": video_id,
                    "categoria": categoria,
                    "topic": topic,
                    "fecha": fecha,
                    "text": text
                })

            break  # si funcionó con esta key, no sigue probando más
        except Exception as e:
            continue

    return comentarios
# 📥 Cargar el log anterior (si existe)
try:
    log = pd.read_csv("log_comentarios_extraidos.csv")
    print("✅ Log cargado con", len(log), "comentarios anteriores.")
except FileNotFoundError:
    log = pd.DataFrame(columns=["comment_id", "video_id", "categoria", "topic", "fecha", "text"])
    print("📁 Log creado desde cero.")

# 🔁 Parámetros ajustables
N_VIDEOS_POR_TOPICO = 50
MAX_COMENTARIOS_POR_VIDEO = 80

todos_comentarios = []

for _, row in df_resumen.iterrows():
    categoria = row["Categoría"]
    archivo = archivos[categoria]
    df_cat = pd.read_csv(archivo)

    for t in range(8):  # Tópico 0 a 7
        col = f"Tópico_{t}"
        if pd.isna(row[col]):
            continue

        df_topico = df_cat[df_cat["topic"] == t]
        if len(df_topico) < N_VIDEOS_POR_TOPICO:
            continue

        videos_muestreados = df_topico.sample(N_VIDEOS_POR_TOPICO, random_state=random.randint(1, 10000))
        for _, video in videos_muestreados.iterrows():
            comentarios = extraer_comentarios(
                video["video_id"], categoria, t, max_results=MAX_COMENTARIOS_POR_VIDEO
            )
            todos_comentarios.extend(comentarios)

# ✅ Combinar nuevos con antiguos (sin repetir)
df_nuevos = pd.DataFrame(todos_comentarios)

# Evitar duplicados antes de guardar
df_total = pd.concat([log, df_nuevos], ignore_index=True)
df_total = df_total.drop_duplicates(subset=["comment_id"])

# Guardar actualizado
df_total.to_csv("log_comentarios_extraidos.csv", index=False)

print(f"✅ Nuevos comentarios agregados: {len(df_nuevos)}")
print(f"🧾 Total acumulado: {len(df_total)} comentarios únicos")


In [ ]:
import os

ruta = "/content/drive/MyDrive/comentarios_emociones"
os.makedirs(ruta, exist_ok=True)
import shutil

shutil.copy("/content/log_comentarios_extraidos.csv", "/content/drive/MyDrive/comentarios_emociones/log_comentarios_extraidos.csv")
archivos = [
    "log_comentarios_extraidos.csv",
    "narrativas_internacional_clusterizadas.csv",
    "narrativas_internacional_medio_oriente_clusterizadas.csv",
    "narrativas_israel_clusterizadas.csv",
    "narrativas_palestina_clusterizadas.csv",
    "resumen_topicos_por_categoria.csv"
]

for archivo in archivos:
    origen = f"/content/{archivo}"
    destino = f"/content/drive/MyDrive/comentarios_emociones/{archivo}"
    shutil.copy(origen, destino)
    print(f"✅ Copiado: {archivo}")
import pandas as pd

archivos_narrativas = {
    "Palestina": "narrativas_palestina_clusterizadas.csv",
    "Israel": "narrativas_israel_clusterizadas.csv",
    "Internacional": "narrativas_internacional_clusterizadas.csv",
    "Internacional Medio Oriente": "narrativas_internacional_medio_oriente_clusterizadas.csv"
}

conteo_videos = []

for categoria, archivo in archivos_narrativas.items():
    df_temp = pd.read_csv(archivo)
    total_videos = df_temp["video_id"].nunique()
    conteo_videos.append({"categoria": categoria, "total_videos": total_videos})

df_conteo = pd.DataFrame(conteo_videos)
df_conteo
import pandas as pd

# Cargar desde Colab
df_comments_test = pd.read_csv("/content/log_comentarios_extraidos.csv")

# Crear muestra balanceada: 8000 comentarios proporcionales por categoría/tópico
grupo = df_comments_test.groupby(["categoria", "topic"])
proporciones = (grupo.size() / grupo.size().sum() * 8000).astype(int).reset_index(name="n_muestra")

df_sample = pd.DataFrame()
for _, row in proporciones.iterrows():
    muestra = df_comments_test[
        (df_comments_test["categoria"] == row["categoria"]) &
        (df_comments_test["topic"] == row["topic"])
    ].sample(n=row["n_muestra"], random_state=42)
    df_sample = pd.concat([df_sample, muestra], ignore_index=True)

# Columnas para etiquetado
df_sample["emocion_sugerida"] = ""
df_sample["emocion_corregida"] = ""

# Guardar muestra en tu Drive
df_sample.to_csv("/content/drive/MyDrive/comentarios_emociones/muestra_etiquetado_ollama.csv", index=False)
print("✅ Muestra de 8000 comentarios lista y guardada en tu Google Drive.")